This notebook continuously estimates the prevailing, realtime, positivity values via 

* $positives_{c, \small{100K}, \tau} \;\; vs.\ \; tests_{c, \small{100K}, \tau}$

curves, wherein

$\qquad \qquad \qquad positives_{c, \small{100K}, \tau} \; = \; \small{100\,000} \times \displaystyle \left(\sum_{\substack{
   start \\
   date
  }}^{\tau}{P_{\delta}}  \right) \times \frac{1}{population}$


$\qquad \qquad \qquad \qquad  tests_{c, \small{100K}, \tau} \; = \; \small{100\,000} \times \displaystyle \left(\sum_{\substack{
   start \\
   date
  }}^{\tau}{T_{\delta}} \right) \times \frac{1}{population}$  

<br>

which are the positive cases per hundred thousand and test numbers per hundred thousand, respectively, based on their cumulative values and

<br>

<table style="width:35%; text-align: left; border: 0px solid black; float:left; margin-left: 50px">
    <tr>
        <th style="width:20%;">$\mathcal{Variable} \qquad$</th><th></th> 
    </tr>
    <tr>
        <td>$\tau$</td><td>end date</td>
    </tr>
    <tr>
        <td>$\delta$</td><td>a series date</td>
    </tr>
    <tr>
      <td>$P_{\delta}$</td><td>The number of positive cases on date $\delta$.</td>
    </tr>
    <tr>
      <td>$T_{\delta}$</td><td>The number of tests on date $\delta$.</td>
    </tr>
    
</table>


<br>

<br>

The subscript $c$ denotes cumulative.   The graph image below illustrates the curves in question.  In brief 

* The quotient of the coordinates of a point along a curve is the cumulative positivity value. 

* The gradient of a modelled secant line to a sequential set of curve rear points estimates the prevailing positivity value.

* The aforementioned secant line model predicts the expected values of $positives_{c, \small{100K}}$ as $tests_{c, \small{100K}}$ increases.


<div style="margin-left: 100px;">
    <img src="https://github.com/briefings/sars/raw/develop/fundamentals/atlantic/docs/ptr.png" longdesc="surveillance" width="40%" style="float:center">
</div>

<br>

Upcoming: Link this image to the interactive graph.



<br>
<br>

## Preliminaries: In Progress

### Libraries

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import dask

import logging

import os
import pathlib
import sys

import statsmodels.regression.linear_model


In [2]:
from statsmodels.formula.api import ols

<br>

### Paths

In [3]:
child = os.getcwd()
parent = str(pathlib.Path(child).parent)

In [4]:
root = os.path.join(child, 'warehouse')
warehouse = os.path.join(root, 'prospects')

<br>

Appending paths

In [5]:
sys.path.append(parent)

<br>

### Logging

In [6]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

<br>

### Custom

Import

In [7]:
import cartographs.boundaries.us.boundaries
import cartographs.boundaries.us.settings

In [8]:
import algorithms.curves.estimates
import algorithms.curves.predictions
import algorithms.curves.secants

In [9]:
import atlantic.base.directories
import atlantic.explore.prospects
import atlantic.explore.hospitalisations

<br>

Set-up directories

In [10]:
directories = atlantic.base.directories.Directories()
directories.cleanup(listof=[warehouse])
directories.create(listof=[warehouse])

<br>
<br>

### Classes

**Boundaries**

In [11]:
settings = cartographs.boundaries.us.settings.Settings()
boundaries = cartographs.boundaries.us.boundaries.Boundaries(settings.crs)

In [12]:
states = boundaries.states(year=settings.latest)
states.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STATEFP   56 non-null     object  
 1   STATENS   56 non-null     object  
 2   AFFGEOID  56 non-null     object  
 3   GEOID     56 non-null     object  
 4   STUSPS    56 non-null     object  
 5   NAME      56 non-null     object  
 6   LSAD      56 non-null     object  
 7   ALAND     56 non-null     int64   
 8   AWATER    56 non-null     int64   
 9   geometry  56 non-null     geometry
dtypes: geometry(1), int64(2), object(7)
memory usage: 4.5+ KB


<br>
<br>

## Data

### Attributes

In [13]:
fields = ['datetimeobject', 'STUSPS', 'positiveIncrease', 'testIncrease', 'deathIncrease', 'hospitalizedIncrease', 
          'positiveCumulative', 'testCumulative',  'deathCumulative', 'hospitalizedCumulative', 
          'positiveRate', 'testRate', 'deathRate', 'hospitalizedRate', 'ndays']

dtype = {'STUSPS': 'str', 'positiveIncrease': np.float64, 'testIncrease': np.float64, 'deathIncrease': np.float64, 
         'hospitalizedIncrease': np.float64, 'positiveCumulative': np.float64, 'testCumulative': np.float64, 
         'deathCumulative': np.float64, 'hospitalizedCumulative': np.float64, 'positiveRate': np.float64, 
         'testRate': np.float64, 'deathRate': np.float64, 'hospitalizedRate': np.float64, 'ndays': np.int64}        

parse_dates = ['datetimeobject']

<br>

### Source URL

In [14]:
datauri = os.path.join(parent, 'warehouse', 'baselines.csv')

<br>

### Data

In [15]:
baselines = pd.read_csv(filepath_or_buffer=datauri, header=0, usecols=fields, 
                        dtype=dtype, encoding='utf-8', parse_dates=parse_dates)

baselines.loc[:, 'positiveTestRate'] = np.where(baselines['testRate'] > 0, 100 * baselines['positiveRate'] / baselines['testRate'], 0)

baselines.loc[:, 'deathPositiveRate'] = np.where(baselines['positiveRate'] > 0, 100 * baselines['deathRate'] / baselines['positiveRate'], 0)

logger.info(baselines.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14924 entries, 0 to 14923
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   datetimeobject          14924 non-null  datetime64[ns]
 1   STUSPS                  14924 non-null  object        
 2   deathIncrease           14924 non-null  float64       
 3   deathCumulative         14924 non-null  float64       
 4   positiveIncrease        14924 non-null  float64       
 5   positiveCumulative      14924 non-null  float64       
 6   hospitalizedIncrease    14924 non-null  float64       
 7   hospitalizedCumulative  14924 non-null  float64       
 8   testIncrease            14924 non-null  float64       
 9   testCumulative          14924 non-null  float64       
 10  deathRate               14924 non-null  float64       
 11  positiveRate            14924 non-null  float64       
 12  testRate                14924 non-null  float6

<br>
<br>

## Regression

### $\small{Set-up}$

#### Latest Date

In [16]:
latestdate = baselines.datetimeobject.max()
logger.info(type(latestdate))
logger.info(latestdate)

INFO:__main__:<class 'pandas._libs.tslibs.timestamps.Timestamp'>
INFO:__main__:2020-11-03 00:00:00


<br>
<br>

#### Periods

Within the context of the aforementioned $positives_{c, \small{100K}, \tau} \;\; vs.\ \; tests_{c, \small{100K}, \tau}$ curves, each value of this array is the number of data points, starting from the end of a curve, that are used to develop **a regression model of the tangent/secant line to the points**.

Key points:

* The model predicts the expected $positives_{c, \small{100K}}$ as $tests_{c, \small{100K}, \tau}$ increases.

* The gradient of each model is an estimate of the prevailing positivity rate; an estimate of the current daily positivity value. [cf. continous positivity value] 

In [17]:
periods = np.arange(8, 29)
logger.info( periods)

INFO:__main__:[ 8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28]


<br>
<br>

#### Partitions

In [18]:
partitionby = 'STUSPS'
partitions = baselines[partitionby].unique()
logger.info(partitions)

INFO:__main__:['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'DE' 'DC' 'FL' 'GA' 'HI' 'ID' 'IL'
 'IN' 'IA' 'KS' 'KY' 'LA' 'ME' 'MD' 'MA' 'MI' 'MN' 'MS' 'MO' 'MT' 'NE'
 'NV' 'NH' 'NJ' 'NM' 'NY' 'NC' 'ND' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD'
 'TN' 'TX' 'UT' 'VT' 'VA' 'WA' 'WV' 'WI' 'WY' 'PR']


<br>
<br>

### **positives vs. tests**

#### Analysis

In [19]:
formula = 'positiveRate ~ testRate'
regressor = 'testRate'

prospects = atlantic.explore.prospects.Prospects(data=baselines, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=100, num=5, latestdate=latestdate)

In [20]:
estimates, predictions = prospects.exc(partitions=baselines[partitionby].unique())

<br>

Estimates

In [21]:
logger.info(estimates.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   datetimeobject           52 non-null     datetime64[ns]
 1   gradient                 52 non-null     object        
 2   lowerconfidenceinterval  52 non-null     object        
 3   upperconfidenceinterval  52 non-null     object        
 4   pvalue                   52 non-null     object        
 5   rsquared                 52 non-null     object        
 6   intercept                52 non-null     object        
 7   STUSPS                   52 non-null     object        
dtypes: datetime64[ns](1), object(7)
memory usage: 3.4+ KB


<br>

Predictions

In [22]:
logger.info(predictions.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   testRate  260 non-null    float64
 1   minimum   260 non-null    float64
 2   median    260 non-null    float64
 3   maximum   260 non-null    float64
 4   STUSPS    260 non-null    object 
dtypes: float64(4), object(1)
memory usage: 10.3+ KB


<br>

#### State of affairs

In [23]:
ptr = estimates

<br>

Prevailing Positive Test Rate

In [24]:
ptr.loc[:, 'prevailingPTR'] = 100 * ptr['gradient']
ptr = ptr.merge(baselines[['datetimeobject', 'STUSPS', 'positiveTestRate']], how='left', on=['datetimeobject', 'STUSPS'])
ptr.loc[:, 'cf'] = ptr[['prevailingPTR', 'positiveTestRate']].max(axis=1)

ptr.loc[:, 'rank'] = ptr['cf'].rank(method='min', ascending=False)
ptr.sort_values(by='rank', ascending=True, inplace=True)
ptr.drop(columns=['cf'], inplace=True)

<br>

Structuring

In [25]:
ptr.loc[:, 'prevailingTPC'] = np.where(ptr['prevailingPTR'] > 0, ptr['prevailingPTR'].rdiv(100), 0)
ptr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 'positiveTestRate': 'continuousPTR'},
          inplace=True)
ptr.to_csv(path_or_buf=os.path.join(warehouse, 'ptr.csv'), index=False, header=True, encoding='utf-8')

ptr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingPTR,continuousPTR,rank,prevailingTPC
32,2020-11-03,0.0144523,0.0128174,0.0159709,1.81279e-22,0.997653,1529.09,NY,1.44523,3.447433,48.0,69.1929
8,2020-11-03,0.0175464,0.0140122,0.0234342,8.47768e-16,0.982367,1152.6,DC,1.75464,3.285222,49.0,56.9917
6,2020-11-03,0.0279106,0.0216646,0.0382092,2.70099e-16,0.986021,202.442,CT,2.79106,3.119011,50.0,35.8287
45,2020-11-03,0.0233941,0.0163405,0.0389403,7.29237e-14,0.967403,-360.86,VT,2.33941,1.176582,51.0,42.7458
19,2020-11-03,0.00882404,0.00644623,0.0140184,1.06193e-12,0.961102,82.7592,ME,0.882404,1.068891,52.0,113.327


<br>
<br>

### **deaths vs. positives**

In [26]:
del formula, regressor, prospects, estimates, predictions

<br>

#### Analysis

In [27]:
formula = 'deathRate ~ positiveRate'
regressor = 'positiveRate'

prospects = atlantic.explore.prospects.Prospects(data=baselines, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=20, num=5, latestdate=latestdate)

In [28]:
estimates, predictions = prospects.exc(partitions=baselines[partitionby].unique())

<br>

#### State of Affairs

In [29]:
dpr = estimates

<br>

Prevailing Death Positive Rate

In [30]:
dpr.loc[:, 'prevailingDPR'] = 100 * dpr['gradient']
dpr = dpr.merge(baselines[['datetimeobject', 'STUSPS', 'deathPositiveRate']], how='left', on=['datetimeobject', 'STUSPS'])
dpr.loc[:, 'cf'] = dpr[['prevailingDPR', 'deathPositiveRate']].max(axis=1)

dpr.loc[:, 'rank'] = dpr['cf'].rank(method='min', ascending=False)
dpr.sort_values(by='rank', ascending=True, inplace=True)
dpr.drop(columns=['cf'], inplace=True)

<br>

Structuring

In [31]:
dpr.loc[:, 'prevailingTPC'] = np.where(dpr['prevailingDPR'] > 0, dpr['prevailingDPR'].rdiv(100), 0)
dpr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 'deathPositiveRate': 'continuousDPR'},
          inplace=True)
dpr.to_csv(path_or_buf=os.path.join(warehouse, 'dpr.csv'), index=False, header=True, encoding='utf-8')

dpr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingDPR,continuousDPR,rank,prevailingTPC
27,2020-11-03,0.00720793,0.00454142,0.00827011,3.40704e-15,0.98097,7.239,NE,0.720793,0.903332,48.0,138.736
49,2020-11-03,0.00797219,0.00674699,0.00975042,2.34033e-18,0.991712,2.95392,WI,0.797219,0.864830,49.0,125.436
50,2020-11-03,0.00607927,0.0019093,0.00798135,4.88657e-12,0.950423,0.778521,WY,0.607927,0.636158,50.0,164.493
44,2020-11-03,0.00307616,0.00255034,0.00390306,4.66879e-17,0.988864,7.88391,UT,0.307616,0.519372,51.0,325.081
1,2020-11-03,0.00341789,0.00256232,0.00791247,6.31895e-08,0.880918,3.23392,AK,0.341789,0.481431,52.0,292.578


<br>
<br>

### **hospitalisations vs. positives**

In [32]:
del formula, regressor, prospects, estimates, predictions

<br>

#### Data

In [33]:
hospitalisations = atlantic.explore.hospitalisations.Hospitalisations(blob=baselines, warehouse=os.path.join(parent, 'warehouse'))

In [34]:
hospital = hospitalisations.exc(limit=periods.min())

INFO:atlantic.explore.hospitalisations:
Hospitalized Positives Curves:
None



<class 'pandas.core.frame.DataFrame'>
Int64Index: 10332 entries, 0 to 10331
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   datetimeobject            10332 non-null  datetime64[ns]
 1   STUSPS                    10332 non-null  object        
 2   hospitalizedRate          10332 non-null  float64       
 3   positiveRate              10332 non-null  float64       
 4   ndays                     10332 non-null  int64         
 5   hospitalizedPositiveRate  10332 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 565.0+ KB


<br>

#### Analysis

In [35]:
formula = 'hospitalizedRate ~ positiveRate'
regressor = 'positiveRate'

prospects = atlantic.explore.prospects.Prospects(data=baselines, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=10, num=5, latestdate=latestdate)

In [36]:
estimates, predictions = prospects.exc(partitions=hospital[partitionby].unique())

<br>

#### State of Affairs

In [37]:
hpr = estimates

<br>

Prevailing Hospitalised Positive Rate

In [38]:
hpr.loc[:, 'prevailingHPR'] = 100 * hpr['gradient']
hpr.loc[:, 'rank'] = hpr['prevailingHPR'].rank(method='min', ascending=False)
hpr.sort_values(by='rank', ascending=True, inplace=True)

<br>

Structuring

In [39]:
hpr = hpr.merge(hospital[['datetimeobject', 'STUSPS', 'hospitalizedPositiveRate']], how='left', on=['datetimeobject', 'STUSPS'])
hpr.loc[:, 'prevailingCPH'] = np.where(hpr['prevailingHPR'] > 0, hpr['prevailingHPR'].rdiv(100), 0)

hpr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 'hospitalizedPositiveRate': 'continuousHPR'}, 
           inplace=True)

hpr.to_csv(path_or_buf=os.path.join(warehouse, 'hpr.csv'), index=False, header=True, encoding='utf-8')

hpr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingHPR,rank,continuousHPR,prevailingCPH
31,2020-11-03,0.0197224,-0.00355323,0.0450631,0.000890414,0.508705,304.621,CT,1.97224,32.0,16.595359,50.7037
32,2020-11-03,0.0195048,0.0139997,0.0237577,1.71274e-16,0.985659,31.5179,WY,1.95048,33.0,3.221835,51.2694
33,2020-11-03,0.0178458,0.0109031,0.021295,6.20804e-16,0.982773,151.542,MA,1.78458,34.0,8.230343,56.0357
34,2020-11-03,0.0132555,0.00949261,0.0172616,6.19232e-14,0.960246,46.4735,NH,1.32555,35.0,6.848358,75.4404
35,2020-11-03,-1.11022e-16,-1.40537e-13,1.44534e-13,0.890842,-535.35,462.615,NY,-1.11022e-14,36.0,17.519355,0


<br>
<br>

## Clean-up

In [40]:
!rm -rf *.log
!rm -rf *.pdf
!rm -rf states